# DCGAN_review  
  
리뷰는 논문의 흐름대로 진행하겠습니다. :)

## Abstract  
  
최근 컴퓨터 비전 분야에서 지도학습의 CNN분야가 굉장히 주목받았다. 반면 비지도 학습의 CNN은 덜 주목받았다. 이 논문에서는 DCGAN을 소개하면서 CNN의 비지도 학습에 대해서 보여주고 있다. 실험을 통해 deep convolutional adversarial pair가 generator와 discriminator 둘 다에서 작은 오브젝트 부터 이미지 전체를 hierarchy하게 학습했다는 것을 보였다. 그리고 여기에 새로운 task에 적용해보면서 적용의 일반성을 확인했다.

## Introduction  
  
최근 학계에서 unlabeled data를 표현하고 사용하는 것에 큰 관심을 보이고 있으며 많은 연구들이 진행 중이다. 이런 문제를 해결할 수 있게 하는 것 중 하나가 GAN이나, GAN은 학습에서 안정적이지 못하며, 학습하는 것을 이해하고 시각화 하는데 제한적이라는 문제가 있다.  
  
그래서 이 연구에서는 다음과 같은 것들을 보여주려고 한다.  
1. Convolutional GAN의 학습에 안정적으로 진행될 수 있게끔 구조적인 topology에 대해서 조건들을 제안하고 평가한다. 이것에 대한 이름을 DCGAN이라고 짓는다.
2. 다른 비지도 학습의 알고리즘의 성능을 비교하면서, 이미지 분류 작업에 학습한 discriminator를 사용한다.
3. GAN에 의해 학습한 필터들을 보여주고, 필터들이 특정 오브젝트를 그리도록 학습한 것을 보여준다.
4. Generator들이 흥미로운 벡터연산을 한다는 점을 보여준다. 이 점은 쉽게 샘플생성에서 의미적인 부분을 만들 수 있는 특성이다.  

## Related Work  
과거에 이 쪽 분야에서 아래와 같은 연구들이 꾸준히 진행됐다.  
  
#### 라벨링이 없는 데이터로부터의 Representation Learning

#### Natural Images 생성

#### CNN 내부 시각화

## Approach and Model Architecture  
  
지금까지의 CNN을 사용한 GAN의 시도는 성공적이지 못했다. 저자들 역시 몇 가지 어려움을 겪었지만 여러 시도를 통해 안정적인 학습을 할 수 있었고, 깊은 모델과 높은 퀄리티를 만들 수 있었다.  
  
논문에서 사용한 중요한 접근 방식은 아래와 같다.

1. pooling layer들을 모두 convolutional net으로 바꾼다.  
        학습 도중 알아서 downsampling, upsampling이 진행된다.  

2. convolutional feature에서 fully connected layers를 모두 제거한다.  
        GAP방식이 가장 대표적인 예이다. 논문에서는 마지막 convolutional feature를 직접 generater와 discriminator의 input과 output을 각각 연결하는 방식을 사용했다.  

3. Batch Normalization을 사용한다.  
        input을 평균을 0, 특정 분산 값을 가지는 정규분포를 따르게 하여 학습이 안정적으로 이루어 지게 한다. 또한 좋지 않은 초기화 문제를 다룰 수 있게하고, gradient flow를 도와준다. collapse를 방지해준다.  
        그러나 모든 레이어에 batchnorm을 진행할 필요는 없다.여기서는 generator output layer와 discriminator input layer에서 batchnorm을 적용하지 않았다.  
        
4. ouput layer에는 Tanh function을, 나머지 layer에서는 ReLU activation을 사용했다.

## Details of Adversarial Training  
  
저자들은 LSUN, Imagenet-1K, Faces 데이터셋에 대하여 DCGAN을 학습시켰다.  
- 이미지를 -1,1로 스케일링 한 것을 제외하고는 아무런 전처리를 하지 않았따.
- 모든 모델은 128 mini batch로 SGD 방식으로 학습했다.
- 모든 가중치는 평균이 0이고, 표준편차가 0.02를 따르게 했다.
- leakyReLU에서 leak의 slope는 모두 0.2로 고정했다.
- hyperparameter 진행 시, adam optimizer를 사용했다.
- 0.001의 학습률은 너무 높아서 0.0002를 사용했다.
- 학습의 안정화를 위해서 $Beta_1$은 0.9에서 0.5로 줄였다.  
  
### LSUN  
  
GAN을 사용해서 image의 퀄리티는 높아졌지만, overfitting과 memorization문제가 발생했다. 저자들은 DCGAN이 더 많은 데이터를 만들고, 높은 복원율을 보여주기 위해서 LSUN 데이터 셋을 모델에 학습시켰다. 저자들은 한 번의 epoch로 얻어진 sample들과 converage한 모델의 sample을 비교했다. 이로 인해 저자들은 DCGAN이 overfitting이나 memorization을 통해 좋은 결과를 도출하는 것이 아님을 알았다.  
  
#### Deduplication  
  input example을 generator가 memorizing하는 가능성을 줄이기 위해 저자들은 image de-duplication 과정을 적용했다. hash-collision의 시각적인 조사를 해보면 100개 중 1개 정도의 false positive가 발생하는 높은 precision을 볼 수 있었다. 이 방법으로 275000개의 이미지를 발견하고 제거했다.  
    
### Faces
저자들은 웹에서 사람 얼굴을 포함한 이미지를 긁어와 데이터셋을 만들었다. 데이터셋은 3M개의 이미지로 구성됐고, OpenCV face detector를 돌려 얼굴 부분만 350000개의 데이터를 얻었다. data augmentation은 진행하지 않았다.  
  
#### Imagenet - 1K
비지도 학습을 위한 자연적인 이미지 source로서 Imagenet-1k를 사용했다. 32 * 32 사이즈로 잘라냈으며 data augmentation은 적용하지 않았다.  
  


## Empirical Validation of DCGANs Capabilities  
  
### GAN을 feature extractor로 사용하여 CIFAR-10을 분류함  

비지도 학습의 퀄리티를 측정하는 흔한 방법은 feature extracotr로 사용해서 지도학습의 결과를 평가하는 것이다. 제일 쉬운 방법은 K-means 방법으로 feature를 뽑아내는 single layer를 사용하는 것이다. 이 때, accuracy는 80.6%를 달성하며 mulit-layer를 사용했을 때는 82%의 accuracy를 도출했다. DCGAN으로 학습한 결과들의 퀄리티를 평가하기 위해, 저자들은 Imagenet-1k를 학습시켰고 discriminator의 모든 레이어에 convolutional feature를 사용했다. 이 feature들은 flatten과 concatenate해서 28672 차원의 벡터를 만들었고, 이를 L2-SVM classifier에 적용했다. 이 방식은 82.8%의 accuracy를 달성했다. 
  
그러나 DCGAN의 방법은 여전히 Exemplar CNN보다 더 낮은 결과치를 보여주지만 좀 더 범용적인 사용이 가능함을 알 수 있었다.  
### GAN을 feature extractor로 사용해 SVHN을 분류함  
label된 데이터가 부족한 SVHN에도 적용해봤다. 전반적인 과정은 위의 내용과 비슷하다. 결과는 22.48% test error를 달성하며 꽤 좋게 나왔다. 이는 CNN을 변형하여 만든 모델보다 더 나은 성능을 보였다. 또한 실험을 통해 DCGAN에서 사용한 CNN architecture가 성능 개선의 핵심이 아님을 증명했다.  


## Investigating And Visualizing The Internals Of The Networks  
  
학습한 generator와 discirminator를 여러 방식으로 조사했다. 

### Walking in the Latent space  
첫 번째로 진행한 실험은 latent space의 전체를 이해하는 것이었다. 학습된 다방면을 걸어보는 것은 memorization 사인을 우리에게 알려주고, space가 계층적으로 collapsed하는 방식도 알 수 있다. latent space를 따라 걷는 것이 image generation에 있어서 의미적인 변화가 발생하면 모델은 흥미로운 표현을 학습한 것이다.  
  
### Visualizing the Discriminator Feature  
비지도 방식을 사용한 DCGAN에 많은 이미지를 학습시키면 계층적으로 feature를 학습하는 것을 발견할 수 있다. 

### Manipulating the Generator Representation  
  
#### 어떤 오브젝트를 그리는 것을 잊게하다.  
지금까지는 discriminator를 바탕으로 논문이 진행됐다. 이 때, generator가 학습한 representations는 무엇일까에 대한 궁금증이 생겼다. 그래서 저자들은 generator에서 창문을 완전히 지워서 실험을 진행했다. 두 번째로 높은 convolution 레이어 feature에서, 로지스틱 회귀를 fit했다. 그래서 창문이 있는 bounding box 내부에는 양의 값을, 이미지의 나머지 부위는 음의 값을 부여했으며 0보다 큰 weight 값은 모두 drop시켰다. (즉, 창문을 모두 제거한 것) 그러고 새로운 sample을 만들어보니 DCGAN은 창문 대신 다른 것을 집어넣었다.

#### 얼굴 sample에 vector 연산하기
space에 벡터 연산을 수행함으로서 매우 풍부한 linear structure를 발견했다. 대표적인 예로 king - man + woman = Queen이라는 결과를 도출하는 것이다. generator의 representation에도 이와 같은 것을 적용해봤다. 하나의 sample에만 적용했을 때에는 굉장히 불안정했으나 3개의 sample을 평균낸 것에서는 의미적인 변화가 보였다. 

## Conclusion and Future Work  
우리는 GAN의 매우 안정적인 학습 구조를 제안했다. 여전히 불안정한 문제점이 남아있으나 이는 미래의 과제로 남기기로 한다. 

## Reference  

https://kakalabblog.wordpress.com/2017/06/04/unsupervised-representation-learning-with-dcgan-2016-%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0/  
https://jgrammer.tistory.com/entry/%EB%85%BC%EB%AC%B8-%EB%A6%AC%EB%B7%B0-DCGAN-2016